In [1]:
# %load ./header.py
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt

import sys

if '../sdfpy' not in sys.path:
  sys.path.insert(0,'../sdfpy')

import sd_sim
import sdfpy as sdf

OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period


In [2]:
%store -r
[A,B,C,D] = the_filter
print(A,B,C,D)

[[-1.07239893e+03 -7.16367470e+07 -3.81029515e+10 -1.26242182e+15]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]] [[1.]
 [0.]
 [0.]
 [0.]] [[-1.07239893e+00  5.75019733e+05 -3.81029515e+07 -1.22070312e-03]] [[0.001]]


## Matrix Balancing
An important step in step in linear algebra computations is matrix balancing. This is the preprocessing of an ill conditioned matrix. The matrix is unbalanced when the L2 norm of the rows / columns are different by orders of magnitude. This leads to computations that are numerically unstable. 

[Matrix Balancing in Lp Norms](https://escholarship.org/uc/item/3847b5dr)

In [3]:
[A, T] = linalg.matrix_balance(A)
B = linalg.solve(T, B) 
C = C @ T

## $\Sigma\Delta$ Modulation
$\Sigma\Delta$ stream computation is done directly on a 2nd order $\Sigma\Delta$ modulated bitstream. The bitstream must be over sampled beyond the Nyquist samle rate to produce data with adequate signal to noise ratio (SNR). The oversampling ratio (OSR) is the integer multiple of increased sampling over the Nyquist frequency. 

The density of ones at the modulator output is proportional to the input signal. A greater number of zeros is generated for decreasing input and more ones for increasing input. The integrator in $\Sigma\Delta$ modulator acts as a lowpass filter for the signal and filters out the high frequency quantization noise. This noise shaping increases the SNR beyond what the oversampling does. 

[Tutorial Sigma-Delta ADCs](https://www.maximintegrated.com/en/design/technical-documents/tutorials/1/1870.html)

In [4]:
OSR = 256      # oversample ratio
fb  = 22050    # nyquist
fs  = OSR*2*fb # sampling frequency
ts  = 1/fs     # sampling period

# State Space Model

- x and $\dot{x}$ are the state vector and the differential state vector respectively
- u and y are input vector and output vector respectively
- A is the system matrix
- B and C are the input and the output matrices
- D is the feed-forward matrix

Below is the continuous time representation

$\dot{x} = A_{c}x + B_{c}u$<br>
$y = C_{c}x + D_{c}u$

Which can also be represented in block matrix form

$\left[\begin{matrix}A & B\\C & D\end{matrix}\right]$ := $C \left(Is - A\right)^{-1} B + D$

[Control Systems - State Space Model](https://www.tutorialspoint.com/control_systems/control_systems_state_space_model.htm)

## Converting from Continuous Time to Sampled Time
The $\delta$-operator is a difference based operator defined as

$\delta x\left(k\Delta\right)\triangleq\frac{x\left(k\Delta+\Delta\right)-x\left(k\Delta\right)}{\Delta}$

Taking the limit of $\delta x\left(k\right)$ as the sampling period approaches zero leads to

$\underset{\Delta\rightarrow0}{lim}\delta x\left(k\Delta\right)=\underset{\Delta\rightarrow0}{lim}\frac{x\left(k\Delta+\Delta\right)-x\left(k\Delta\right)}{\Delta}=\frac{dx}{dt}$

Which is continuous time differentiation

The corresponding discrete $\delta$ based state space representation

$\delta$x = $A_{\delta}x + B_{\delta}u$<br>
$y = C_{\delta}x + D_{\delta}u$

With the state space matrices defined below 

${\Delta} = ts = \dfrac{1}{fs}$, sampling period

$A_{\delta}=\frac{e^{A_{c}\Delta}-I}{\Delta}$

$B_{\delta}=\frac{1}{\Delta}\int_{0}^{\Delta}e^{A_{c}\left(t-\tau\right)}B_{c}u\left(\tau\right)d\tau$

$C_{\delta}=C_{c}$

$D_{\delta}=D_{c}$

$\begin{array}{c}
H_{\delta}(\delta)=C_{\delta}(\delta I-A_{\delta})^{-1}B_{\delta}+D_{\delta}\end{array}=\frac{N\left(\delta\right)}{D\left(\delta\right)}$

Where

$N\left(\delta\right)=\beta_{\delta0}+\beta_{\delta1}\delta^{-1}+\cdots+\beta_{\delta n-1}\delta^{-\left(n-1\right)}+\beta_{\delta n}\delta^{-n}\\=\sum_{i=0}^{N}\beta_{\delta i}\delta^{-i}$

$D\left(\delta\right)=1+\alpha_{\delta1}\delta^{-1}+\cdots+\alpha_{\delta n-1}\delta^{-\left(n-1\right)}+\alpha_{\delta n}\delta^{-n}\\=1+\sum_{i=1}^{N}\alpha_{\delta i}\delta^{-i}$

[Digital Control and Estimation](https://dl.acm.org/doi/10.5555/574885)

In [5]:
Ad = (linalg.expm(A*ts) - np.eye(A.shape[0])) / ts
Bd = ((linalg.expm(A*ts) - np.eye(A.shape[0]) ) @ B) / ts
Bd = np.linalg.inv(A) @ Bd
Cd = C
Dd = D

print(Ad)
print(Bd)
print(Cd)
print(Dd)

[[-1.07552054e+03 -8.74451015e+03 -1.13716863e+03 -9.18489055e+03]
 [ 8.19161017e+03 -3.17263682e+00 -4.12383224e-01 -3.33243951e+00]
 [ 1.48602997e+00  4.09599962e+03 -4.98588946e-05 -4.03019313e-04]
 [ 1.79717825e-04  7.43038514e-01  4.09600000e+03 -3.76019216e-08]]
[[9.53628934e-07]
 [3.45993316e-10]
 [4.18438169e-14]
 [3.67427569e-18]]
[[-1.12449178e+06  7.36025259e+07 -1.19071724e+06 -9.31322575e-09]]
[[0.001]]


# Structural Transformation of Filter
![$\delta$DFIIt Filter Architecture](./Figures/dfiit.PNG)

The structure of the $\Sigma\Delta$ stream computation is based on a generalized direct-form delta operator-based IIR filter ($\delta$DFIIt). The $\delta$DFIIt is not just limited to filters but can also be used for state space realization of control applications. 

[A generalized direct-form delta operator-based IIR filter with minimum noise gain and sensitivity](https://ieeexplore.ieee.org/document/933811)

A similarity transform of the sampled time state space matrix is evaluated. This puts the $\delta$DFIIt structure into the observable canonical form, $T_{0}$. $T_{1}$ is first calculated as below.

$T_{1}=\left[\begin{array}{c}
C_{\delta}\\
C_{\delta}A_{\delta}\\
\vdots\\
C_{\delta}A_{\delta}^{n-1}
\end{array}\right]^{-1}\left[\begin{array}{c}
0\\
0\\
\vdots\\
1
\end{array}\right]$

In [6]:
e     = np.zeros((A.shape[1], 1))
e[-1] = 1

O = control.obsv(A, C)
[U, S, Vh] = linalg.svd(O)
V = Vh.T
S = np.diag(S)

S_inv = linalg.solve(S, np.eye(S.shape[0]))
T_inv = V @ S_inv @ U.conj().T
T1 = T_inv @ e;

The the observable canonical is now calculated

$T_{0}=\left[\begin{array}{ccccc}
A_{\delta}^{n-1}T_{1} & A_{\delta}^{n-2}T_{1} & \ldots & A_{\delta}T_{1} & T_{1}\end{array}\right]$

In [7]:
n  = A.shape[1]
T0 = np.zeros((n,n))

for i in range(1, n+1):
  column = np.linalg.matrix_power(A, n-i) @ T1
  T0[:, i-1] = column[:,0]

$T_{0}$ is now in the $\delta$DFIIt form. This similarity transformation produces the state space representation below

$\tilde{A_{\delta}}=T_{0}^{-1}A_{\delta}T_{0}$

$\tilde{B_{\delta}}=T_{0}^{-1}B_{\delta}$

$\tilde{C_{\delta}}=C_{\delta}T_{0}$

In [8]:
Ad_t = linalg.solve(T0, A @ T0)
Ad_t[:-1,1:] = np.eye(n-1)
Ad_t[-1,1:] = 0
Bd_t = linalg.solve(T0, B)
Cd_t = C @ T0;
Dd_t = D;

print(Ad_t)
print(Bd_t)
print(Cd_t)
print(Dd_t)

[[-1.07239893e+03  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-7.16367470e+07  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-3.81029515e+10  0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-1.26242182e+15  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[-1.07239893e+00]
 [ 5.75019988e+05]
 [-3.82395575e+07]
 [ 9.08452912e+06]]
[[ 1.00000000e+00  5.61489063e-20 -2.02184553e-20 -1.88183984e-16]]
[[0.001]]


## $\Sigma\Delta$ Transfer Function

The $\Sigma\Delta$ Transfer Function is used repeatedly in the following calculations. The *delta_bode()* function is used to convert from continuous state space representation to the discrete $\delta$ based state space representation.

$\large \delta I = ((e^{\frac{-j\omega}{\Delta}} - 1) \times \frac{1}{\Delta})\times I$

$H(\delta)=\tilde{C_{\delta}}(\delta I-\tilde{A_{\delta}})\tilde{B_{\delta}}+D_{\delta}$

In [9]:
def delta_bode(A,B,C,D,f,ts):
  q     = C.shape[0]
  p     = B.shape[1]
  fs    = 1/ts
  mag   = np.zeros((q,p,f.shape[0]))
  phz   = np.zeros((q,p,f.shape[0]))
  delta = (np.exp(1j*2*np.pi*(f/fs))-1)/ts

  for i in range(f.shape[0]):
      A_d = delta[i] * np.eye(A.shape[0]) - A
      
      [A_d, T_d] = linalg.matrix_balance(A_d)
      B_d = linalg.solve(T_d, B)
      C_d = C @ T_d
      
      h   = linalg.solve(A_d, B_d)
      h   = (C_d @ h) + D
      mag[:,:,i] = np.abs(h)
      phz[:,:,i] = 180*np.arctan2(np.imag(h),np.real(h))/np.pi
  return mag, phz

## Dynamic Range Scaling of State Variable Integrators

The internal integrators must be scaled appropriately to maximize the SNR at each node. A scaling matrix $T_{s}$  scales the coefficients $k_{1}$ to $k_{N}$ located after the $\delta^{-1}$ operators (integrators).

$T_{s}=diag\left[k_{1}^{-1},\left(k_{1}k_{2}\right)^{-1},\ldots,\left(k_{1}k_{2}\ldots k_{n}\right)^{-1}\right]$

The state space equations are then transformed

$\tilde{A_{\delta}^{'}}=T_{s}^{-1}T_{0}^{-1}A_{\delta}T_{0}T_{s}$

$\tilde{B_{\delta}^{'}}=T_{s}^{-1}T_{0}^{-1}B_{\delta}$

$\tilde{C_{\delta}^{'}}=C_{\delta}T_{0}T_{s}$

$H(\delta)=\tilde{C_{\delta}^{'}}(\delta I-\tilde{A_{\delta}^{'}})\tilde{B_{\delta}^{'}}+D_{\delta}$

Transfer functions from the input to the ith state integrator

$f_{i}\left(\delta\right)=\frac{x_{i}\left(\delta\right)}{u\left(\delta\right)}$ = $T_{s}^{-1}T_{0}^{-1}\left(\delta I-A_{\delta}\right)^{-1}B_{\delta}$

In [10]:
f = np.logspace(0,np.log10(fb),2**10)
T0_inv = linalg.solve(T0, np.eye(T0.shape[0]))
[f_i, phz] = sdf.delta_bode(Ad,Bd,T0_inv,0,f,ts)

The internal gain of the filter structure is calculated with the $\infty$-norm for discrete $\delta$ based filters to normalize each integrator node. The scaling coefficients in $T_{s}$ are then be found using back substitution.

$\left\Vert H(\frac{e^{j\omega}-1}{\Delta})\right\Vert _{p}=\left[\frac{1}{2\pi}\intop_{-\pi}^{\pi}|H\left(\frac{e^{j\omega}-1}{\Delta}\right)|^{p}d\omega\right]^{1/p}$

$||f(\delta)||_{\infty}	=T_{s}^{-1}||T_{0}^{-1}\left(\delta I-A_{\delta}\right)^{-1}B_{\delta}||_{\infty}
	=\left[\begin{array}{ccc}
1 & \cdots & 1\end{array}\right]^{T}$

In [11]:
f_norm = np.zeros(Ad.shape[0])

for i in range(f_norm.shape[0]):
  f_norm[i] = linalg.norm(f_i[i], np.inf, axis=1)

Ts = np.zeros(Ad.shape)
k = np.zeros(f_norm.shape[0])
k_inv = np.zeros(f_norm.shape[0])

for i in range(f_norm.shape[0]):
  if i == 0:
    k[i] = 1/f_norm[i]
  else:
    k[i] = 1/(np.prod(k[:i])*f_norm[i])

  k_inv[i] = 2**np.floor(np.log2(ts/k[i]))/ts
  Ts[i,i] = np.prod(k_inv[0:i+1])

$\Sigma\Delta$ stream computation modifies the $\delta$DFIIt form by implementing the scaling coefficients as bit shift rather than multiplication. Adjusting the scaling coefficients to be a power of two multiply/divide reduces the filter architecture complexity.

$\tilde{k}_{i}=\frac{2^{\left\lfloor log_{2}\Delta\cdot k_{i}^{-1}\right\rfloor }}{\Delta}$

In [12]:
k_inv = np.diag(k_inv)
[num_t, den_t] = signal.ss2tf(Ad_t,Bd_t,Cd_t,Dd_t)
num_ts = num_t.copy()
num_ts[0,1:] /= np.diag(Ts).T
den_ts = den_t.copy()
den_ts[1:] /= np.diag(Ts).T
den_ts[0] = 1

beta  = num_ts[0]
alpha = den_ts

print(beta)
print(alpha)

[ 1.00000000e-03  3.37894812e-13  1.74327689e+02 -1.11932309e-12
  1.11995359e+01]
[1.00000000e+00 1.59366749e+03 1.93120597e+04 1.86338568e+03
 1.11995359e+04]


In [13]:
the_delta_filter = [Ad,Bd,Cd,Dd,k,k_inv,T0,Ts,f,ts,alpha,beta]
%store the_delta_filter

Stored 'the_delta_filter' (list)


In [14]:
# filter = sdf.sd_filter(OSR,fb)
# filter.run(A,B,C,D)
# sd_sim.sim_filter(filter)

Goto the next step - [Coefficient Sensitivity Analysis](./3_SD_dIIR_sensitivity.ipynb)